In [381]:
class GET_DATA(object):
    def __init__(self):
        global logging
        import logging

    
    def Peace(self):
        """
        This is a function to pull the most updated peace data modified from https://www.kaggle.com/kretes/gpi2008-2016

        Global Peace Index (GPI) measures the relative position of nations' and regions' peacefulness.
        The GPI ranks 163 independent states and territories (99.7 per cent of the world’s population)
        according to their levels of peacefulness. In the past decade, the GPI has presented trends of 
        increased global violence and less peacefulness.

        The lower the number indicates more peace in the region.
        """
        import requests
        import re
        from bs4 import BeautifulSoup
        response = requests.get(url='https://en.wikipedia.org/wiki/Global_Peace_Index')
        soup = BeautifulSoup(response.text, 'html.parser')
        base_year = 2018        #latest year
        years = 9               #number of years to get data
        def get_countries_by_gpi():
            i = 1
            for table in soup.find_all('table', re.compile('wikitable sortable')):
                if table.find_all('th')[0].get_text() == 'Country\n':
                    for tr in table.find_all('tr'):
                        country_name = tr.find_all('a')[0].get_text()
                        if not country_name.startswith('['):
                            row = {'country': country_name}
                            for year, index in zip(range(base_year-years+1,base_year+1),range(2*(years),0,-2)):
                                score = tr.find_all('td')[index].get_text()
                                if score != '' and score != '\n':
                                    row['score_%s' % year] = float(score)
                            yield row

        import pandas as pd
        gpi = pd.DataFrame.from_dict(list(get_countries_by_gpi()))
        # print(gpi)
        gpi.to_csv('gpi_%s-%s.csv' % (base_year-years+1,base_year),index=False)
        return(gpi)    

    
    def Trade(self):
        """
        WITS Trade Stats is a database created by aggregating data from UN COMTRADE and UNCTAD TRAINS database. 
        It provides information on bilateral trade exports, imports and tariffs for over 180 countries and regions.
        """        
        import logging
        logging.basicConfig(filename="test.log", level=logging.DEBUG)
        file_path = "590PR_final_datasets"
        file_name = "wits_en_trade_summary_allcountries_allyears.zip"
        zf = zipfile.ZipFile(file_path + '/' + file_name) 
        df = []
        for name in zipfile.ZipFile.infolist(zf):
            logging.debug(name.filename)
            try:
                df.append(pd.read_csv(zf.open(name.filename), header=0 ))
            except:
                pass
        frame = pd.concat(df, axis=0, ignore_index=True)
        return frame
    
    
    def Hunger(self):
        file_path = "590PR_final_datasets"
        file_name = "Hunger.csv"
        import pandas as pd
        df_hunger = pd.read_csv(file_path + "/" + file_name, na_values = '\t', sep = '\t', header = 0)
        #display(df_hunger)
        list = []
        list.append(df_hunger['Country Name'])
        list.append(df_hunger['Indicator Name'])
        for i in range(2009, 2019):
            year = str(i)
            df = df_hunger[year]
            list.append(df)
        df_new_hunger = pd.concat(list,axis = 1)
        return df_new_hunger
    
    
    def Unemployment(self):
        file_path = "590PR_final_datasets"
        file_name = "unemployment.zip"
        df = pd.read_csv(file_path + '/' + file_name, compression = 'zip')
        return df

    
    def Suicide(self):
        file_path = "590PR_final_datasets"
        file_name = "suicide-rates-overview-1985-to-2016.zip"
        df = pd.read_csv(file_path + '/' + file_name, compression = 'zip')
        return df


    def Freedom(self):
        """The Human Freedom Index presents the state of human freedom in the world based on a broad measure that 
        encompasses personal, civil, and economic freedom. Human freedom is a social concept that recognizes the 
        dignity of individuals and is defined here as negative liberty or the absence of coercive constraint. 
        Because freedom is inherently valuable and plays a role in human progress, it is worth measuring carefully. The Human Freedom Index is a resource that can help to more objectively observe relationships between freedom and other social and economic phenomena, as well as the ways in which the various dimensions of freedom interact with one another.
        """
        file_path = "590PR_final_datasets"
        file_name = "the-human-freedom-index.zip"
        df = pd.read_csv(file_path + '/' + file_name, compression = 'zip')
        return df

    
    def Happiness(self):
        import zipfile
        file_path = "590PR_final_datasets"
        file_name = "world-happiness-report.zip"
        zf = zipfile.ZipFile(file_path + '/' + file_name) 
        df = []
        happy = {}
        for name in zipfile.ZipFile.infolist(zf):
            happy[name.filename] = pd.read_csv(zf.open(name.filename))
        return happy

    
    def GDELT(self, start, end):
        """
        The GDELT Project is the largest, most comprehensive, and highest resolution open database of human society ever created. 
        Just the 2015 data alone records nearly three quarters of a trillion emotional snapshots and more than 1.5 billion location 
        references, while its total archives span more than 215 years, making it one of the largest open-access spatio-temporal 
        datasets in existance and pushing the boundaries of "big data" study of global human society. Its Global Knowledge Graph 
        connects the world's people, organizations, locations, themes, counts, images and emotions into a single holistic network 
        over the entire planet.
        
        This is Gdelt 2.0 (supports from the date after Feb, 2015)
        This function pull the events ids and the country it takes place
        Please input the date in the format of 'yyyy mm dd'.
        
        This function use a package from https://github.com/linwoodc3/gdeltPyR
        """
        import gdelt
        gd = gdelt.gdelt(version=2)
        results = gd.Search([start,end],table='events',coverage=True,translation=False)
        return (results[['GLOBALEVENTID','ActionGeo_FullName']])

    
    def UNODC_crime(self, sheet = None):
        """
        Crime data from UNODC including 
        'Car theft', 'Burglary', 'Robbery', 'Sexual Assault', 'Physical Assault', 'Bribery', 'Police Reporting Car Theft',
        'Police Reporting Burglary', 'Police Reporting Robbery', 'Police Reporting Sexual Assault', 'Police Reporting Bribery'
        or type 'ALL' to get all data.
        """
        import pandas as pd
        from pandas import read_excel
#         sheet = input("Please choose crime report from the following categories\n'Car theft', 'Burglary', 'Robbery', 'Sexual Assault', 'Physical Assault', 'Bribery', 'Police Reporting Car Theft','Police Reporting Burglary', 'Police Reporting Robbery', 'Police Reporting Sexual Assault', 'Police Reporting Bribery'")         
        if sheet == "ALL":
            sheet = None
        file_path = "590PR_final_datasets"
        file_name = "data-and-metadata-from-crime-victimisation-surveys-2004-2014-xlsx-1.xlsx"
        self.dfs = pd.read_excel(file_path + "/" + file_name, sheetname = sheet, header = 12)
#         logging.debug(self.dfs)
#         logging.debug(self.dfs.keys())
        return self.dfs

    def Poverty(self, sheet = None):
        import zipfile
        file_path = "590PR_final_datasets"
        file_name = "PovStats_csv.zip"
        zf = zipfile.ZipFile(file_path + '/' + file_name) 
        df = []
        pov = {}
        for name in zipfile.ZipFile.infolist(zf):
#             logging.debug(name.filename)
            pov[name.filename] = pd.read_csv(zf.open(name.filename))
#         logging.debug((df))
        if not sheet == None:
            return pov[sheet]
        else:
            return pov

    def Married(self, sheet = None):
        """
        Marriged data from The United Nations Statistics Division.
        http://data.un.org/Data.aspx?q=Marital&d=POP&f=tableCode%3A23
        """
        import pandas as pd
        from pandas import read_excel
#         sheet = input("Please choose crime report from the following categories\n'Car theft', 'Burglary', 'Robbery', 'Sexual Assault', 'Physical Assault', 'Bribery', 'Police Reporting Car Theft','Police Reporting Burglary', 'Police Reporting Robbery', 'Police Reporting Sexual Assault', 'Police Reporting Bribery'")         
        if sheet == "ALL":
            sheet = None
        file_path = "590PR_final_datasets"
        file_name = "UNPD_WMD_2017_MARITAL_STATUS.xlsx"
        self.dfs = pd.read_excel(file_path + "/" + file_name, sheetname = sheet, header = 2)
#         logging.debug(self.dfs)
#         logging.debug(self.dfs.keys())
        return self.dfs




In [ ]:
# dir(GET_DATA)

freedom = GET_DATA().Freedom()
gdelt = GET_DATA().GDELT("2016 10 15", "2016 10 16")
happiness = GET_DATA().Happiness()
hunger = GET_DATA().Hunger()
married = GET_DATA().Married('CURRENTLY MARRIED')
peace = GET_DATA().Peace()
# print(GET_DATA().Poverty().keys())
# GET_DATA().Poverty("PovStatsData.csv")
poverty = GET_DATA().Poverty()
suicide = GET_DATA().Suicide
trade = GET_DATA().Trade
crime = GET_DATA().UNODC_crime()
burglary = GET_DATA().UNODC_crime('Burglary')
unemploy = GET_DATA().Unemployment()



In [395]:
hunger = GET_DATA().Hunger()

hunger.rename(columns={'Country Name':'country'}, inplace=True)

peace = GET_DATA().Peace()


In [426]:
married['AgeGroup']=='Total'

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
49405    False
49406    False
49407    False
49408    False
49409    False
49410    False
49411    False
49412    False
49413    False
49414    False
49415    False
49416    False
49417    False
49418    False
49419    False
49420    False
49421    False
49422    False
49423    False
49424    False
49425    False
49426    False
49427    False
49428    False
49429    False
49430    False
49431    False
49432    False
49433    False
49434    False
Name: AgeGroup, Length: 49435, dtype: bool

In [422]:
married.groupby(['Country or area']).mean()

,ISO code,YearStart,YearEnd,AgeStart,AgeEnd,DataValue,DataCatalog ID,Including_consensual_unions
Country or area,,,,,,,,
Afghanistan,4.0,1996.200000,1997.110000,37.900000,97.500000,73.162900,3079.960000,NaN
Albania,8.0,2003.275862,2003.551724,38.931034,93.827586,68.063241,2975.986207,1.0
Algeria,12.0,1999.308642,1999.512346,40.679012,101.777778,62.535864,2518.333333,NaN
American Samoa,16.0,1981.703704,1981.703704,43.703704,115.851852,63.505741,2267.962963,NaN
Angola,24.0,2000.104478,2000.701493,38.059701,97.432836,64.713731,3757.597015,1.0
Anguilla,660.0,2001.000000,2001.000000,42.500000,112.214286,39.500000,2544.000000,NaN
Antigua and Barbuda,28.0,1996.416667,1996.416667,42.708333,123.791667,44.105208,1414.416667,1.0
Argentina,32.0,1991.500000,1991.500000,41.400000,119.400000,59.392700,1850.560000,1.0
Armenia,51.0,2007.752066,2007.867769,35.537190,85.735537,63.544545,3821.834711,1.0


In [418]:
married[['Country or area', 'YearStart', 'DataValue']]

,Country or area,YearStart,DataValue
0,Afghanistan,1972,7.60
1,Afghanistan,1972,31.90
2,Afghanistan,1972,59.10
3,Afghanistan,1972,78.90
4,Afghanistan,1972,87.20
5,Afghanistan,1972,89.70
6,Afghanistan,1972,90.70
7,Afghanistan,1972,91.50
8,Afghanistan,1972,91.50
9,Afghanistan,1972,85.40


In [433]:
def population():
    file_path = "590PR_final_datasets"
    file_name = "API_SP.POP.TOTL_DS2_en_csv_v2_10576638.csv"
    df = pd.read_csv(file_path + '/' + file_name, header= 4)
    return df

population()

,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351,9166764,9345868,9533954,9731361,9938414,...,28803167,29708599,30696958,31731688,32758020,33736494,34656032,35530081,Unnamed: 62,Unnamed: 63
0,Angola,AGO,"Population, total",SP.POP.TOTL,5.643182e+06,5.753024e+06,5.866061e+06,5.980417e+06,6.093321e+06,6.203299e+06,...,2.336913e+07,2.421856e+07,2.509615e+07,2.599834e+07,2.692047e+07,2.785930e+07,2.881346e+07,2.978419e+07,NaN,NaN
1,Albania,ALB,"Population, total",SP.POP.TOTL,1.608800e+06,1.659800e+06,1.711319e+06,1.762621e+06,1.814135e+06,1.864791e+06,...,2.913021e+06,2.905195e+06,2.900401e+06,2.895092e+06,2.889104e+06,2.880703e+06,2.876101e+06,2.873457e+06,NaN,NaN
2,Andorra,AND,"Population, total",SP.POP.TOTL,1.341100e+04,1.437500e+04,1.537000e+04,1.641200e+04,1.746900e+04,1.854900e+04,...,8.444900e+04,8.375100e+04,8.243100e+04,8.078800e+04,7.922300e+04,7.801400e+04,7.728100e+04,7.696500e+04,NaN,NaN
3,Arab World,ARB,"Population, total",SP.POP.TOTL,9.249093e+07,9.504450e+07,9.768229e+07,1.004111e+08,1.032399e+08,1.061750e+08,...,3.565089e+08,3.648959e+08,3.733070e+08,3.817021e+08,3.900430e+08,3.983050e+08,4.064527e+08,4.144919e+08,NaN,NaN
4,United Arab Emirates,ARE,"Population, total",SP.POP.TOTL,9.263400e+04,1.010780e+05,1.124720e+05,1.255660e+05,1.385290e+05,1.503620e+05,...,8.270684e+06,8.672475e+06,8.900453e+06,9.006263e+06,9.070867e+06,9.154302e+06,9.269612e+06,9.400145e+06,NaN,NaN
5,Argentina,ARG,"Population, total",SP.POP.TOTL,2.061908e+07,2.095308e+07,2.128768e+07,2.162184e+07,2.195393e+07,2.228339e+07,...,4.122389e+07,4.165688e+07,4.209674e+07,4.253992e+07,4.298152e+07,4.341776e+07,4.384743e+07,4.427104e+07,NaN,NaN
6,Armenia,ARM,"Population, total",SP.POP.TOTL,1.874120e+06,1.941491e+06,2.009526e+06,2.077575e+06,2.144998e+06,2.211316e+06,...,2.877311e+06,2.875581e+06,2.881922e+06,2.893509e+06,2.906220e+06,2.916950e+06,2.924816e+06,2.930450e+06,NaN,NaN
7,American Samoa,ASM,"Population, total",SP.POP.TOTL,2.001300e+04,2.048600e+04,2.111700e+04,2.188200e+04,2.269800e+04,2.352000e+04,...,5.563700e+04,5.532000e+04,5.523000e+04,5.530700e+04,5.543700e+04,5.553700e+04,5.559900e+04,5.564100e+04,NaN,NaN
8,Antigua and Barbuda,ATG,"Population, total",SP.POP.TOTL,5.533900e+04,5.614400e+04,5.714400e+04,5.829400e+04,5.952400e+04,6.078100e+04,...,9.466100e+04,9.571900e+04,9.677700e+04,9.782400e+04,9.887500e+04,9.992300e+04,1.009630e+05,1.020120e+05,NaN,NaN
9,Australia,AUS,"Population, total",SP.POP.TOTL,1.027648e+07,1.048300e+07,1.074200e+07,1.095000e+07,1.116700e+07,1.138800e+07,...,2.203175e+07,2.234002e+07,2.273346e+07,2.312813e+07,2.347569e+07,2.381600e+07,2.419091e+07,2.460186e+07,NaN,NaN


In [493]:
def marital():
    file_path = "590PR_final_datasets"
    file_name = "UNdata_MARITAL_STATUS_2010-2013.csv"
    df = pd.read_csv(file_path + '/' + file_name)
    df.drop(df.tail(74).index,inplace=True)
    
    #     df.dropna(inplace=True)

    file_name = "UNdata_MARITAL_STATUS_2014-2017.csv"
    df2 = pd.read_csv(file_path + '/' + file_name)
    df2.drop(df2.tail(54).index,inplace=True)

    df_all = pd.concat([df, df2], axis= 0).reset_index(drop=True)
    return df_all

In [676]:
married = marital()
married = married.astype({"Year": int}, copy = False)
# married['Year'].astype('category').values.categories
married[married['Marital status']=='Total']

C:\Users\worawich-win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,Country or Area,Year,Area,Sex,Age,Marital status,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Åland Islands,2013,Total,Male,Total,Total,Estimate - de jure,"Final figure, complete",2014.0,1.425500e+04,1
5,Åland Islands,2013,Total,Male,0 - 14,Total,Estimate - de jure,"Final figure, complete",2014.0,2.424000e+03,1
10,Åland Islands,2013,Total,Male,15 - 19,Total,Estimate - de jure,"Final figure, complete",2014.0,8.400000e+02,1
15,Åland Islands,2013,Total,Male,20 - 24,Total,Estimate - de jure,"Final figure, complete",2014.0,8.620000e+02,1
20,Åland Islands,2013,Total,Male,25 - 29,Total,Estimate - de jure,"Final figure, complete",2014.0,8.000000e+02,1
25,Åland Islands,2013,Total,Male,30 - 34,Total,Estimate - de jure,"Final figure, complete",2014.0,9.080000e+02,1
30,Åland Islands,2013,Total,Male,35 - 39,Total,Estimate - de jure,"Final figure, complete",2014.0,9.240000e+02,1
35,Åland Islands,2013,Total,Male,40 - 44,Total,Estimate - de jure,"Final figure, complete",2014.0,9.400000e+02,1
40,Åland Islands,2013,Total,Male,45 - 49,Total,Estimate - de jure,"Final figure, complete",2014.0,1.053000e+03,1
45,Åland Islands,2013,Total,Male,50 - 54,Total,Estimate - de jure,"Final figure, complete",2014.0,9.570000e+02,1


In [669]:
married['Year'].astype('category').values.categories

Index([  2010,   2011,   2012,   2013, '2010', '2011', '2012', '2013', '2014',
       '2015', '2016', '2017'],
      dtype='object')

In [655]:
married.astype({"Year": int})

,Country or Area,Year,Area,Sex,Age,Marital status,Record Type,Reliability,Source Year,Value,Value Footnotes
0,Åland Islands,2013,Total,Male,Total,Total,Estimate - de jure,"Final figure, complete",2014.0,1.425500e+04,1
1,Åland Islands,2013,Total,Male,Total,Single (never married),Estimate - de jure,"Final figure, complete",2014.0,7.863000e+03,1
2,Åland Islands,2013,Total,Male,Total,Married,Estimate - de jure,"Final figure, complete",2014.0,5.062000e+03,1
3,Åland Islands,2013,Total,Male,Total,Widowed and not remarried,Estimate - de jure,"Final figure, complete",2014.0,3.020000e+02,1
4,Åland Islands,2013,Total,Male,Total,Divorced and not remarried,Estimate - de jure,"Final figure, complete",2014.0,1.029000e+03,1
5,Åland Islands,2013,Total,Male,0 - 14,Total,Estimate - de jure,"Final figure, complete",2014.0,2.424000e+03,1
6,Åland Islands,2013,Total,Male,0 - 14,Single (never married),Estimate - de jure,"Final figure, complete",2014.0,2.424000e+03,1
7,Åland Islands,2013,Total,Male,0 - 14,Married,Estimate - de jure,"Final figure, complete",2014.0,0.000000e+00,1
8,Åland Islands,2013,Total,Male,0 - 14,Widowed and not remarried,Estimate - de jure,"Final figure, complete",2014.0,0.000000e+00,1
9,Åland Islands,2013,Total,Male,0 - 14,Divorced and not remarried,Estimate - de jure,"Final figure, complete",2014.0,0.000000e+00,1


In [541]:
m2 = married.loc[(married['Marital status']=='Total') & (married['Age']=='Total')]

m2[m2['Year'] == 2013].groupby(['Country or Area']).sum()

,Source Year,Value
Country or Area,,
Azerbaijan,4028.0,9416801.0
Canada,4030.0,35154279.0
Costa Rica,4030.0,4711986.0
Czechia,4026.0,10516125.0
Denmark,4026.0,5608784.0
Faeroe Islands,4030.0,48092.0
Finland,4028.0,5410388.5
France,4030.0,63786141.0
Germany,4026.0,82020578.0


In [683]:
married = marital()
married = married.astype({"Year": int}, copy = False)

# percent_married = pd.DataFrame()
percent_married = []
for i in range(8):
    total = married.loc[(married['Marital status']=='Total') & (married['Age']=='Total')].groupby(['Year', 'Country or Area'], as_index=False).sum()
    single = married.loc[(married['Marital status']=='Single (never married)') & (married['Age']=='Total')].groupby(['Year', 'Country or Area'], as_index=False).sum()
    single_pop = single[single['Year'] == 2010+i].groupby(['Year', 'Country or Area']).sum()['Value']
    total_pop = total[total['Year'] == 2010+i].groupby(['Year', 'Country or Area']).sum()['Value']
    percent_married.append(((total_pop-single_pop)/total_pop))
percent_married

C:\Users\worawich-win\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2903: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


[Year  Country or Area                                     
 2010  Argentina                                               0.480960
       Aruba                                                   0.482935
       Bahamas                                                 0.391690
       Barbados                                                0.334383
       Denmark                                                 0.530201
       Finland                                                 0.526326
       France                                                  0.508185
       Georgia                                                      NaN
       Greenland                                               0.364365
       Hungary                                                 0.574902
       Iceland                                                 0.534270
       Israel                                                  0.495046
       Italy                                                   0.588717
    

In [ ]:
percent_married

In [665]:
married_pop = married.loc[(married['Marital status']=='Married') & (married['Age']=='Total')].groupby(['Year', 'Country or Area'], as_index=False, axis = 0).sum()
married_pop[(married_pop)['Year'] == 2014]

,Year,Country or Area,Source Year,Value


In [574]:
(married_pop/total_pop).to_frame()

Value
Year Country or Area             
2010 Argentina           0.357639
     Aruba               0.347216
     Bahamas             0.268441
     Barbados            0.238447
     Denmark             0.393980
     Finland             0.374070
     France              0.382435
     Greenland           0.268446
     Hungary             0.387085
     Iceland             0.325066
     Israel              0.403646
     Italy               0.494352
     Latvia              0.372014
     Liechtenstein       0.453448
     Lithuania           0.410859
     Malaysia            0.431377
     Malta               0.492393
     Netherlands         0.418501
     Norway              0.355228
     Philippines         0.352362
     Puerto Rico         0.384400
     Romania             0.495496
     Russian Federation  0.403595
     San Marino          0.533732
     Seychelles          0.190962
     Slovakia            0.437377
     Slovenia            0.397941
     Sweden              0.340160
     Åland Islands       0.358910
2011 Albania             0.518891
...                           ...
2016 Slovenia            0.383727
     Sweden              0.333828
     Switzerland         0.428563
     Åland Islands       0.347022
2017 Bulgaria            0.415912
     Canada              0.387985
     Costa Rica          0.272997
     Czechia             0.398137
     Denmark             0.370721
     Estonia             0.314472
     Faeroe Islands      0.374263
     Finland             0.362089
     Hungary             0.359577
     Iceland             0.323429
     Italy               0.476693
     Latvia              0.366544
     Liechtenstein       0.453645
     Lithuania           0.414340
     Luxembourg          0.402140
     Netherlands         0.381138
     Norway              0.343458
     San Marino          0.501062
     Slovakia            0.407676
     Slovenia            0.381315
     Sweden              0.332891
     Switzerland         0.425810
     Åland Islands       0.345451
2010 Georgia                  NaN
2012 Austria                  NaN
2015 Malta                    NaN

[250 rows x 1 columns]

In [560]:
single_pop

Year  Country or Area      
2010  Argentina                3.130482e+07
      Aruba                    5.247400e+04
      Bahamas                  2.260280e+05
      Barbados                 1.505580e+05
      Denmark                  2.604479e+06
      Finland                  7.619533e+06
      France                   3.096438e+07
      Greenland                7.181750e+04
      Hungary                  4.250992e+06
      Iceland                  1.481050e+05
      Israel                   3.849549e+06
      Italy                    2.487581e+07
      Latvia                   9.519850e+05
      Liechtenstein            4.662000e+04
      Lithuania                1.358565e+06
      Malaysia                 1.502587e+07
      Malta                    1.803860e+05
      Netherlands              7.701812e+06
      Norway                   2.469810e+06
      Philippines              5.185422e+07
      Puerto Rico              1.090047e+06
      Romania                  8.139423e+06
    

In [563]:
married[married['Country or Area'] == 'Georgia']

,Country or Area,Year,Area,Sex,Age,Marital status,Record Type,Reliability,Source Year,Value,Value Footnotes
23439,Georgia,2010,Total,Male,Total,Total,Estimate - de facto,"Final figure, complete",2011.0,2118100.0,NaN
23440,Georgia,2010,Total,Male,0 - 14,Total,Estimate - de facto,"Final figure, complete",2011.0,399500.0,NaN
23441,Georgia,2010,Total,Male,15 - 19,Total,Estimate - de facto,"Final figure, complete",2011.0,169600.0,NaN
23442,Georgia,2010,Total,Male,20 - 24,Total,Estimate - de facto,"Final figure, complete",2011.0,185300.0,NaN
23443,Georgia,2010,Total,Male,25 - 29,Total,Estimate - de facto,"Final figure, complete",2011.0,174800.0,NaN
23444,Georgia,2010,Total,Male,30 - 34,Total,Estimate - de facto,"Final figure, complete",2011.0,158200.0,NaN
23445,Georgia,2010,Total,Male,35 - 39,Total,Estimate - de facto,"Final figure, complete",2011.0,150500.0,NaN
23446,Georgia,2010,Total,Male,40 - 44,Total,Estimate - de facto,"Final figure, complete",2011.0,140400.0,NaN
23447,Georgia,2010,Total,Male,45 - 49,Total,Estimate - de facto,"Final figure, complete",2011.0,154300.0,NaN
23448,Georgia,2010,Total,Male,50 - 54,Total,Estimate - de facto,"Final figure, complete",2011.0,142500.0,NaN


In [398]:
import math
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
def Analysis_first_two_level(hunger, peace): 
    df_level1 = pd.merge(hunger, peace, on = 'country', how='inner')
    df_level1 = df_level1.drop_duplicates(keep='first', inplace=False)
    x_list = []
    y_list = []
    item = [['undernourishment_rate_2010','pi_2010'],['undernourishment_rate_2011','pi_2011'],
              ['undernourishment_rate_2012','pi_2012'],['undernourishment_rate_2013','pi_2013'],
              ['undernourishment_rate_2014','pi_2014'],['undernourishment_rate_2015','pi_2015'],
              ['undernourishment_rate_2016','pi_2016']]
    for i in item:
        df = df_level1[i]
        dd = df.sort_values(by=i[0], ascending=True)
        x = np.asarray(dd[i[0]])
        y = np.asarray(dd[i[1]])
        for j in range(x.shape[0]):
            if math.isnan(x[j]) == True:
                new_x = np.delete(x,j)
                new_y = np.delete(y,j)
        x_list.append(new_x)
        y_list.append(new_y)
    return x_list, y_list

In [399]:
%matplotlib inline
import matplotlib.pyplot as plt
def plot(x_list, y_list):
    plt.figure(figsize = (30,15), dpi=100)
    item = [['undernourishment_rate_2010','pi_2010'],['undernourishment_rate_2011','pi_2011'],
                  ['undernourishment_rate_2012','pi_2012'],['undernourishment_rate_2013','pi_2013'],
                  ['undernourishment_rate_2014','pi_2014'],['undernourishment_rate_2015','pi_2015'],
                  ['undernourishment_rate_2016','pi_2016']]
    marker = ['.','*','>','<','1','2','s']
    color = ['#E11B00', '#1E90FF','#FF4233','#FFE333','#7EFF33','#33F4FF','#D433FF']
    for f in range(len(x_list)):
        plt.plot(x_list[f],y_list[f],label = 'data of year '+item[f][1][-4:], marker = marker[f],color = color[f])
    plt.xlabel('undernourishment rate')
    plt.ylabel('Peacefulness Index')
    plt.ylim(1,4)
    plt.xticks(fontsize = 8, horizontalalignment = 'center', alpha = .7)
    plt.yticks(fontsize = 12, alpha = .7)
    plt.grid(axis='both',alpha = .3)
    plt.legend()
    plt.show()

In [400]:
x_list, y_list = Analysis_first_two_level(hunger, peace)

KeyError: "None of [Index(['undernourishment_rate_2010', 'pi_2010'], dtype='object')] are in the [columns]"

In [401]:
plot(x_list, y_list)

NameError: name 'x_list' is not defined

In [402]:
%matplotlib inline
import matplotlib.pyplot as plt
def plot_cat(x_list, y_list):
    plt.figure(figsize = (30,20), dpi=100)
    item = [['undernourishment_rate_2010','pi_2010'],['undernourishment_rate_2011','pi_2011'],
                  ['undernourishment_rate_2012','pi_2012'],['undernourishment_rate_2013','pi_2013'],
                  ['undernourishment_rate_2014','pi_2014'],['undernourishment_rate_2015','pi_2015'],
                  ['undernourishment_rate_2016','pi_2016']]
    x_item = ['[0,5)','[5,10)','[10,15)','[15,20)','[20,25)','[25,30)','[30,35)','[35,40)','[40,45)',
             '[45,50)','[50+']
    marker = ['.','*','>','<','1','2','s']
    color = ['#E11B00', '#1E90FF','#FF4233','#FFE333','#7EFF33','#33F4FF','#D433FF']
    y_new_list = []
    for a in range(len(x_list)):
        cc = x_list[a]
        cy = y_list[a]
        y_item = np.zeros((11,2))
        for b in range(len(cc)):
            if cc[b] >= 0 and cc[b]<5:
                y_item[0][0] += cy[b]
                y_item[0][1] += 1
            if cc[b] >= 5 and cc[b]<10:
                y_item[1][0] += cy[b]
                y_item[1][1] += 1
            if cc[b] >= 10 and cc[b]<15:
                y_item[2][0] += cy[b]
                y_item[2][1] += 1
            if cc[b] >= 15 and cc[b]<20:
                y_item[3][0] += cy[b]
                y_item[3][1] += 1
            if cc[b] >= 20 and cc[b]<25:
                y_item[4][0] += cy[b]
                y_item[4][1] += 1
            if cc[b] >= 25 and cc[b]<30:
                y_item[5][0] += cy[b]
                y_item[5][1] += 1
            if cc[b] >= 30 and cc[b]<35:
                y_item[6][0] += cy[b]
                y_item[6][1] += 1
            if cc[b] >= 35 and cc[b]<40:
                y_item[7][0] += cy[b]
                y_item[7][1] += 1
            if cc[b] >= 40 and cc[b]<45:
                y_item[8][0] += cy[b]
                y_item[8][1] += 1
            if cc[b] >= 45 and cc[b]<50:
                y_item[9][0] += cy[b]
                y_item[9][1] += 1
            if cc[b] >= 50:
                y_item[10][0] += cy[b]
                y_item[10][1] += 1
        y_new = np.zeros(y_item.shape[0])
        for c in range(y_item.shape[0]):
            y_new[c] = y_item[c][0]/y_item[c][1]
        y_new_list.append(y_new)
    for s in range(len(y_new_list)):
        plt.plot(x_item,y_new_list[s],label = 'data of year '+item[s][1][-4:], marker = marker[s],color = color[s])
    plt.xlabel('undernourishment rate')
    plt.ylabel('Peacefulness Index')
    plt.ylim(1.5,3.5)
    plt.xticks(fontsize = 8, horizontalalignment = 'center', alpha = .7)
    plt.yticks(fontsize = 12, alpha = .7)
    plt.grid(axis='both',alpha = .3)
    plt.legend()
    plt.show()

In [403]:
plot_cat(x_list, y_list)

NameError: name 'x_list' is not defined

In [404]:
j = [['undernourishment_rate_2010','pi_2010'],'undernourishment_rate_2011',
              'undernourishment_rate_2012','undernourishment_rate_2013',
              'undernourishment_rate_2014','undernourishment_rate_2015','undernourishment_rate_2016']

In [405]:
df_steal.sort_values(by='undernourishment_rate_2010', ascending=True)

NameError: name 'df_steal' is not defined

In [406]:
dd = df_steal[['undernourishment_rate_2014','pi_2014']].sort_values(by='undernourishment_rate_2014', ascending=True)
x = dd['pi_2014']
x = np.asarray(x)
x

NameError: name 'df_steal' is not defined